#  Output forcing field

In [1]:
import numpy as np
import pandas as pd

## Definitions

In [ ]:
def pres(r):
    return pn+dph*np.exp(-(rmaxh/r)**bh)

In [ ]:
def hvel2d(l1,l2):
    r=np.sqrt(l1**2+l2**2)
    xh=0.5+(r-rmaxh)/(maxR-rmaxh)*kh
    ur=(bh/rhoa*(rmaxh/r)**bh*dph*np.exp(-(rmaxh/r)**bh))**xh
    theta=np.arctan2(l2,l1)
    ux=-ur*np.sin(theta)
    uy=ur*np.cos(theta)
    return ux+vtx,uy+vty, pres(r)

## Read OutData file

In [ ]:
tc=pd.read_csv('tmp/outData.txt',delimiter='\t')

In [ ]:
# From lat lon to r, th

In [ ]:
xl=np.linspace(-maxR, maxR,1000)
yl=np.linspace(-maxR, maxR,1000)
q1,q2=np.meshgrid(xl,yl)
vtx=-4.6
vty=3.35

In [ ]:
zx,zy, pr=hvel2d(q1,q2)
vel=np.sqrt(zx**2+zy**2)